In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import urllib
from requests import get
from requests.exceptions import RequestException
from contextlib import closing

import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import lxml.html as lh

In [2]:
url='https://www.exchangerates.org.uk/GBP-USD-spot-exchange-rates-history-2019.html'#Create a handle, page, to handle the contents of the website

page = requests.get(url)#Store the contents of the website under doc

doc = lh.fromstring(page.content)#Parse data that are stored between <tr>..</tr> of HTML

tr_elements = doc.xpath('//tr')

td_elements = doc.xpath('//td')

### Data for 1 year

In [3]:
def usd_gbp_new():

    url='https://www.exchangerates.org.uk/GBP-USD-spot-exchange-rates-history-2019.html'
    #Create a handle, page, to handle the contents of the website

    page = requests.get(url)#Store the contents of the website under doc

    doc = lh.fromstring(page.content)

    tr_elements = doc.xpath('//tr') #Parse data that are stored between <tr>..</tr> of HTML
    
    day = [] #Create an empty list to store the data between <tr>....</tr> tags 
    
    for t in tr_elements:
        if len(t) == 3:
            day.append(t.text_content())
            
    tmp = pd.DataFrame(day)
    tmp.columns = ['Data']
    
    tmp['Date'] = tmp['Data'].apply(lambda x : x.split("ay")[-1].split("£1")[0])
# filter((lambda x: re.search(r'L', x)),s)
    tmp['Date'] = pd.to_datetime(tmp['Date'])
    tmp['Rate'] = tmp['Data'].apply(lambda x : x.split("$")[-1].split("British")[0])
    tmp.drop(['Data'],axis = 1,inplace= True)
# tmp.index = tmp['Date']
# tmp['Rate'] = tmp.index
    
#     tmp['Rate'] = int(tmp['Rate'])
#     tmp.reindex(index=tmp.index[::-1])
#     tmp.reset_index(inplace = True)
    
    
    return(tmp)

In [4]:
ex_data = usd_gbp_new()

### Data for last 180 days

In [5]:
def get_data():
    source=urllib.request.urlopen('https://www.exchangerates.org.uk/GBP-USD-exchange-rate-history.html')
    html=source.read()
    
    soup = BeautifulSoup(html,"lxml")
    #(soup.prettify())
    soup.a
    
    data_table=soup.find('table',{"id":'hist'})
    #print (data_table)
    
    day=[]
    rate=[]
    
    for counter,row in enumerate(data_table.findAll('tr')):
        cells=row.findAll('td')
        if len(cells)==3:
            day.append(((cells[0].find(text=True)).split(None,1))[1])
            rate.append(float(((cells[1].find(text=True)).split(' '))[3]))
            
    date=[]
    for counter,object_ in enumerate(day):
        date.append(pd.datetime.strptime(object_, "%d %B %Y"))
        
    rate_data=pd.DataFrame({'Day':date,'Rate':rate})
    rate_data=rate_data.set_index('Day')
    rate_data_full=rate_data.reindex(index=rate_data.index[::-1])
    
    return rate_data_full

In [6]:
rate_data = get_data()